# 对话式 RAG - Conversational RAG
在许多问答应用程序中，我们希望允许用户进行来回对话，这意味着应用程序需要某种过去问题和答案的“记忆”，以及将这些问题和答案纳入当前思维的逻辑。

在本指南中，我们重点介绍如何添加逻辑来整合历史消息。此处介绍了有关聊天记录管理的更多详细信息。

我们将介绍两种方法：

 - 链 Chains，我们始终在其中执行检索步骤；

 - 代理 Agents，我们让 LLM 自行决定是否以及如何执行检索步骤（或多个步骤）。
   

In [1]:
import os
from dotenv import load_dotenv,find_dotenv

_ = load_dotenv(find_dotenv())

In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="http://api.baichuan-ai.com/v1",
    api_key=os.environ["BAICHUAN_API_KEY"],
    model="Baichuan4",
)

In [13]:
from langchain_community.embeddings import BaichuanTextEmbeddings

embeddings = BaichuanTextEmbeddings(baichuan_api_key=os.environ["BAICHUAN_API_KEY"])

In [14]:
# 安装zhipu
# pip install zhipuai

In [15]:
# from langchain_community.embeddings import ZhipuAIEmbeddings
# # import langchain_community
# # langchain_community.embeddings.__all__

# embeddings = ZhipuAIEmbeddings()

In [16]:
import bs4

from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter


loader = WebBaseLoader(
    web_paths=("https://ouzhoubei.co/article-794-1.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(id=("article_content"))
    )
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits,embedding=embeddings)
retriever = vectorstore.as_retriever()

system_prompt = (
    "您是问答任务的助手。"
    "使用以下检索到的上下文来回答问题。如果您不知道答案，请说您不知道。最多使用三句话并保持答案简洁。"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system",system_prompt),
    ("human","{input}")
])

question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [17]:
response = rag_chain.invoke({"input": "意大利队有多少位球员?"})
response["answer"]

'意大利队共有26位球员。'

请注意，我们使用了内置链构造函数 create_stuff_documents_chain 和 create_retrieval_chain，因此我们的解决方案的基本要素是：

 - retriever
 - prompt
 - LLM
   
这将简化合并聊天历史记录的过程。

## 添加聊天记录

我们构建的链直接使用输入查询来检索相关上下文。但在对话设置中，用户查询可能需要对话上下文才能被理解。例如，考虑以下交流：

> 人类：“什么是任务分解？”
>
> 人工智能：“任务分解涉及将复杂的任务分解为更小、更简单的步骤，以便代理或模型更易于管理。”
>
> 人类：“常见的分解方法有哪些？”

为了回答第二个问题，我们的系统需要理解“它”指的是“任务分解”。  
我们需要更新现有应用的两件事：  

 - Prompt 提示：更新我们的提示以支持历史消息作为输入。

 - Contextualizing questions 情境化问题：添加一个子链，该子链接受最新的用户问题并在聊天历史的上下文中重新表述它。这可以简单地被认为是构建一个新的“历史感知”检索器。  
   而之前我们有：

   -  query查询 -> retriever检索器
     
      现在我们将有：
      
   - （query查询，conversation history对话历史）-> LLM大模型 -> rephrased query重新表述的查询 -> retriever检索器
  
**将问题情境化 Contextualizing the question**  

首先，我们需要定义一个子链，该子链接收历史消息和最新的用户问题，并且如果问题引用了历史信息中的任何信息，则重新表述该问题。  

我们将使用一个提示模板，其中包含一个名为 "chat_history" 的 MessagesPlaceholder 变量。  
这使我们能够使用 "chat_history" 输入键向提示传递消息列表，这些消息将在系统消息之后和包含最新问题的人类消息之前插入。 

请注意，我们利用辅助函数 create_history_aware_retriever 来完成此步骤，该函数管理 chat_history 为空的情况，否则按顺序应用 prompt | llm | StrOutputParser() | 检索器。  

create_history_aware_retriever 构造一个链，该链接受键 input 和 chat_history 作为输入，并具有与检索器相同的输出模式。

In [18]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "根据聊天记录和用户最新的问题，"
    "该问题可能涉及聊天历史中的上下文，"
    "形成一个独立的问题，"
    "这个问题是可以在不了解聊天历史的情况下被理解的。"
    "不要回答这个问题，只需在需要时进行重新表述，否则保持原样返回。"
)

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system",contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human","{input}")
])

history_aware_retriever = create_history_aware_retriever(llm,retriever,contextualize_q_prompt)

此链将在我们的检索器前添加输入查询的改写，从而让检索过程融入对话的上下文信息。

现在我们可以构建完整的QA问答链条了。这个过程很简单，只需将检索器更新为我们新创建的history_aware_retriever即可。

同样，我们将使用create_stuff_documents_chain来生成一个question_answer_chain，  
其输入键包括context（上下文）、chat_history（聊天历史）和input（输入）——它接收检索到的上下文，连同对话历史和查询一起，来生成答案。

我们通过create_retrieval_chain来构建最终的RAG（检索增强生成）链条。这条链条依次应用history_aware_retriever和question_answer_chain，为了方便起见，会保留诸如检索到的上下文等中间输出。  
该链条的输入键包括input（输入）和chat_history（聊天历史），其输出则包含input（输入）、chat_history（聊天历史）、context（上下文）和answer（答案）。

In [19]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages([
    ("system",system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human","{input}")
])

question_answer_chain = create_stuff_documents_chain(llm,qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever,question_answer_chain)

让我们尝试一下。下面我们将提出一个问题以及一个需要结合上下文才能给出合理回答的跟进问题。  
由于我们的链条包含了一个"chat_history"（聊天历史）输入，调用者需要管理这个聊天历史。  
我们可以通过将输入和输出的消息追加到一个列表中来实现这一点：

In [20]:
from langchain_core.messages import AIMessage,HumanMessage

chat_history = []

question = "意大利队的球员信息以及所在俱乐部?"
ai_msg_1 = rag_chain.invoke({
    "input":question,
    "chat_history":chat_history
})

chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=ai_msg_1["answer"])
])

question2 = "其中有几位来自尤文图斯俱乐部？"
ai_msg_2 = rag_chain.invoke({
    "input":question2,
    "chat_history":chat_history
})

print(ai_msg_2["answer"])

有两位来自尤文图斯俱乐部，分别是21号：法乔利和24号：坎比亚索。


**聊天记录的状态管理**  

这里我们介绍了如何添加应用程序逻辑来整合历史输出，但我们仍在手动更新聊天记录并将其插入到每个输入中。  
在真正的问答应用程序中，我们需要某种方式来保存聊天记录，以及某种方式来自动插入和更新它。